In [46]:
from PIL import Image
import pytesseract
import cv2
import spacy
from spacy.matcher import Matcher
from rake_nltk import Rake
import os
import docx
import PyPDF2
import numpy as np 
import pandas as pd 
#from PIL import Image

import sys
import nltk
import spacy
import re
import requests
import subprocess
import pandas as pd
from tabulate import tabulate
#from pdfminer.high_level import extract_text
#from PyPDF2 import PdfReader, PdfFileWriter, PdfFileMerger


# Load Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Update with your Tesseract installation path

nlp = spacy.load('en_core_web_sm')

from nanonets import NANONETSOCR

In [31]:
#extract_text_from_pdf: This function takes a PDF file path as input and extracts text from all the pages in the PDF using PyPDF2.

def extract_text_from_pdf_or_image(pdf_path):
    model = NANONETSOCR()
    model.set_token('87dcd5e7-53e9-11ee-bb5e-c26e11187d6d')
    text = model.convert_to_string(pdf_path,formatting='lines') 
    return text

# def extract_text_from_pdf(pdf_path):
#     with open(pdf_path, 'rb') as file:
#         pdf_reader = PyPDF2.PdfReader(file)
#         text = ''
#         for page in pdf_reader.pages:
#             text += page.extract_text()
#     return text

#extract_text_from_docx: This function takes a DOCX file path as input and extracts text from the document using the docx library.
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    text = '\n'.join([para.text for para in doc.paragraphs])
    return text

# #extract_text_from_image: This function takes an image file path as input, converts the image to grayscale, and then uses pytesseract to perform Optical Character Recognition (OCR) and extract text from the image.
# #Load the resume image using OpenCV and use OCR to extract text from the image
# def extract_text_from_image(image_path):
#     # Load the image using OpenCV
#     image = cv2.imread(image_path)
#     # Convert the image to grayscale
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     # Perform OCR to extract text
#     text = pytesseract.image_to_string(gray)
#     return text

In [32]:
#================Extracting person's name==============================================

def extract_names(txt):
    person_names = []
    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )
    return person_names[0]


#========================Extracting phone number========================================== 
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
 
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG, resume_text)
 
    if phone:
        number = ''.join(phone[0])
 
        if resume_text.find(number)  >= 0 and len(number) < 16:
            return number
    return None


#========================Extracting email==========================================

EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
 
def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)


#========================Extracting education=========================================

RESERVED_WORDS = [
    'school',
    'college',
    'university',
    'academy',
    'faculty',
    'degree',
    'institute',]
 
def extract_education(input_text):
    organizations = []
 
    # first get all the organization names using nltk
    for sent in nltk.sent_tokenize(input_text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'ORGANIZATION':
                organizations.append(' '.join(c[0] for c in chunk.leaves()))
 
    # we search for each bigram and trigram for reserved words
    # (college, university etc...)
    education = set()
    for org in organizations:
        for word in RESERVED_WORDS:
            if org.lower().find(word) >= 0:
                education.add(org)
 
    return education 


#========================Extracting skills================================================
SKILLS_DB = [
    'andriod developer',
    'app developer',
    'Javascript',
    'Java',
    'machine learning',
    'data science',
    'python',
    'CSS',
    'doctor',
    'teacher',
    'web development',
    'communication',
    'team work',
]

def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)
    
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
 
    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
    
    # generate bigrams and trigrams (such as artificial intelligence)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
 
    # we create a set to keep the results in.
    found_skills = set()

    # we search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in SKILLS_DB:
            found_skills.add(token)
 
    # we search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS_DB:
            found_skills.add(ngram)
    return found_skills



In [36]:
def parse_resume(resume_text, output_file_path):
#def parse_resume(resume_text):
    name = extract_names(resume_text)
    email = extract_emails(resume_text)
    phone = extract_phone_number(resume_text)
    skills = extract_skills(resume_text)
    education = extract_education(resume_text)
    
 # Replace empty lists with a single empty string
    name = name or ['']
    email = email or ['']
    phone = phone or ['']
    skills = skills or ['']
    education = education or ['']
    info = {
        "Name": name,
        "Email": email,
        "Phone": phone,
        "Skills": skills,
        "education":education
    }
    print (info)
    with open(output_file_path, 'a') as f:
        f.write(tabulate(info, headers="keys"))
        f.write("\n\n")
    

In [45]:
if __name__ == '__main__':
    # Directory containing candidate resumes (PDF, DOCX, and images)
    directory = 'C:/Users/user/Documents/GOMYCODE/ResumeParsing/dataset' 
    for filename in os.listdir(directory):
        output_file_path = 'C:/Users/user/Documents/GOMYCODE/ResumeParsing/output.txt'
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path) and (file_path.lower().endswith('.pdf') or file_path.lower().endswith('.png') or file_path.lower().endswith('.jpg'))  :
            #resume_text = extract_text_from_pdf(file_path)
            resume_text=extract_text_from_pdf_or_image(file_path)
            print('File_name:', filename)
            #print(resume_text)
            parse_resume(resume_text,output_file_path)
        elif os.path.isfile(file_path) and file_path.lower().endswith('.docx'):
            resume_text = extract_text_from_docx(file_path)
            print('-------------File_name:-------------', filename)
            #print(resume_text)
            parse_resume(resume_text ,output_file_path)
            
        # elif os.path.isfile(file_path) and file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        #     resume_text = extract_text_from_image(file_path)
        #     print('**************File_name:*****************', filename)
        #     parse_resume(resume_text ,output_file_path)
        else:
            print('Unsupported File Format:', filename)
        continue
    

File_name: 1.pdf
{'Name': 'Avishek Kumar Data Scientist Machine Learning', 'Email': ['aviw.upes01@gmail.com'], 'Phone': '+91999900011 05', 'Skills': {'Python', 'Machine Learning'}}
File_name: 10.pdf
{'Name': 'Economics Moscow Advanced Machine Learning', 'Email': [''], 'Phone': '91 863465923 3', 'Skills': {'Python', 'team work', 'Machine Learning', 'python', 'communication', 'Data Science'}}
File_name: 11.pdf
{'Name': 'Khan Enthusiastic', 'Email': ['fulkarkhan9980@gmail.com'], 'Phone': '8443405050', 'Skills': {'machine learning', 'data science', 'Python', 'Data Science', 'Machine Learning'}}
File_name: 12.pdf
{'Name': 'Kunika Bhargav Data Science', 'Email': ['bhargav496@gmail.com'], 'Phone': '7983924436', 'Skills': {'Python', 'Data Science', 'Machine Learning'}}
File_name: 13.pdf
{'Name': 'Face API', 'Email': ['gkapil8019@gmail.com'], 'Phone': '9522330896', 'Skills': {'machine learning', 'data science', 'Python', 'Data Science', 'DATA SCIENCE'}}
File_name: 14.pdf
{'Name': 'Machine', 'Em